# Emoji-Style Chatbot Fine-Tuning

## Setup and Configuration
I’m about to take you on a quick, hands-on tour of fine-tuning an OpenAI model using the official Python SDK. Think of this notebook as my little lab where I load a training file, start a job, keep an eye on its progress, and then give the customized model a quick test drive to see how it behaves on real prompts.

In this very first cell I set the stage. I import the SDK client along with a few helpers for reading environment variables, pausing between status checks, and capturing secrets without printing them on screen. Then I name the essentials that guide the entire run: the path to my `.jsonl` training data, the base model I plan to adapt, a short suffix that will help me spot the resulting model later, and the number of epochs that control how many passes the trainer makes over my data. I run this once to pin down the configuration so the rest of the notebook can flow smoothly from upload to job creation to monitoring and finally to a quick sanity check of the tuned model.

In [ ]:
import os
import time
from openai import OpenAI
from getpass import getpass

# Training data file
TRAINING_FILE_PATH = "data/emoji_ft_train.jsonl"

# Model for fine-tuning
BASE_MODEL = "gpt-3.5-turbo"

# Custom suffix to model name
MODEL_SUFFIX = "emoji-v1"

# Set the number of training epochs
N_EPOCHS = 3

## Initialize OpenAI Client
Before I can talk to the API, I need my credentials. I first check the environment for `OPENAI_API_KEY`, and if it is missing I quietly prompt for it with `getpass` so the secret never appears in the notebook output. With the key in hand, I create an `OpenAI` client that all later steps will reuse for uploads, job creation, and status checks. The short print line is my quick confirmation that authentication is set and the SDK is ready to make requests.

In [2]:
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    api_key = getpass("Enter your OpenAI API key: ")

client = OpenAI(api_key=api_key)

print("OpenAI client initialized.")

OpenAI client initialized.


## Upload Training File
Now I hand the API my training data. I open the `.jsonl` file in binary mode and send it to the Files endpoint with the purpose set to fine-tune, which lets the service know this upload is destined for a training job. If everything goes well, I print a friendly confirmation along with the server-assigned file ID and the original filename so I can reference or audit it later. If the path is wrong or the file is missing, the `FileNotFoundError` branch gives me a clear message that points straight to the problematic location, making it easy to fix the path and rerun the cell.

In [3]:
try:
    with open(TRAINING_FILE_PATH, "rb") as f:
        training_file = client.files.create(file=f, purpose="fine-tune")
    print("Training file uploaded successfully!")
    print(f"   ID: {training_file.id}")
    print(f"   Name: {training_file.filename}")
except FileNotFoundError:
    print(f"Error: Training file not found at '{TRAINING_FILE_PATH}'")

Training file uploaded successfully!
   ID: file-NoPS4m5fKQPo7ry41HCsQJ
   Name: emoji_ft_train.jsonl


## Create Fine-Tuning Job
With the training file safely uploaded, I kick off the actual fine-tuning run by creating a job on the service. I pass the server’s file ID along with the base model I want to adapt, set the number of epochs through the hyperparameters, and add a short suffix so the resulting model name is easy to recognize later. The API returns a job object, so I stash its ID for the monitoring step and print a quick snapshot of the job ID and its initial status as a breadcrumb in the notebook output.

In [ ]:
job = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    model=BASE_MODEL,
    hyperparameters={"n_epochs": N_EPOCHS},
    suffix=MODEL_SUFFIX,
)

job_id = job.id

print("Fine-tuning job created!")
print(f"   Job ID: {job_id}")
print(f"   Status: {job.status}")

Fine-tuning job created!
   Job ID: ftjob-m6dEhtrjGqtAFuuKWPuvEkFA
   Status: validating_files


## Monitor Job Progress
Now I settle in to watch the run. I set a polling interval and start a simple timer so each status line shows an elapsed clock in minutes and seconds, which makes progress feel tangible. In the loop I retrieve the job, read its current status, and print a timestamped update. If the service reports that the job has succeeded, failed, or been cancelled, I exit the loop and announce the final state. After that I fetch the definitive job record and try to grab the `fine_tuned_model` identifier. If it is present, I print the new model name so I can use it in the next step; if not, I dump the job details to help me diagnose what went wrong before I try again.

In [5]:
POLL_INTERVAL = 30  # seconds
start_time = time.time()

print(f"Monitoring job {job_id}. Polling every {POLL_INTERVAL} seconds...")

while True:
    job = client.fine_tuning.jobs.retrieve(job_id)
    status = job.status

    elapsed_time = int(time.time() - start_time)
    minutes, seconds = divmod(elapsed_time, 60)

    print(f"[{minutes:02d}:{seconds:02d}] Job Status: {status}")

    if status in ["succeeded", "failed", "cancelled"]:
        print(f"\nJob finished with status: {status}")
        break

    time.sleep(POLL_INTERVAL)

# Retrieve the final job details
final_job = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = final_job.fine_tuned_model

if fine_tuned_model_id:
    print(f"\nFine-tuned model created: {fine_tuned_model_id}")
else:
    print("\nFine-tuned model ID not found. Check job details below:")
    print(final_job)

Monitoring job ftjob-m6dEhtrjGqtAFuuKWPuvEkFA. Polling every 30 seconds...
[00:00] Job Status: validating_files
[00:30] Job Status: validating_files
[01:01] Job Status: validating_files
[01:31] Job Status: running
[02:01] Job Status: running
[02:31] Job Status: running
[03:01] Job Status: running
[03:32] Job Status: running
[04:02] Job Status: running
[04:32] Job Status: running
[05:02] Job Status: running
[05:32] Job Status: running
[06:03] Job Status: running
[06:33] Job Status: running
[07:03] Job Status: running
[07:33] Job Status: running
[08:03] Job Status: running
[08:34] Job Status: running
[09:04] Job Status: running
[09:34] Job Status: running
[10:04] Job Status: running
[10:34] Job Status: running
[11:05] Job Status: running
[11:35] Job Status: running
[12:05] Job Status: running
[12:35] Job Status: running
[13:05] Job Status: running
[13:36] Job Status: running
[14:06] Job Status: running
[14:36] Job Status: running
[15:06] Job Status: running
[15:36] Job Status: running
[1

## View Job Events
With the job wrapped up, I take a quick look under the hood by pulling the event stream for this run. I ask the API for up to fifty recent events, then reverse the list so I can read the story in chronological order from the earliest messages to the latest. Each line shows when the event happened, the severity level, and a short message, which makes it easy to spot data validation notes, training phase changes, or any warnings that might explain delays. I keep the whole thing inside a try and except so a transient API hiccup or a permissions snag does not derail the notebook; if fetching fails, I print the error and move on armed with whatever information I already have.

In [6]:
print(f"Fetching events for job {job_id}...\n")
try:
    # Using the SDK to list events
    events = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=50)

    for event in reversed(list(events.data)):
        print(f"- {event.created_at}: [{event.level}] {event.message}")
except Exception as e:
    print(f"Could not fetch events: {e}")

Fetching events for job ftjob-m6dEhtrjGqtAFuuKWPuvEkFA...

- 1760753005: [info] Step 1665/1707: training loss=0.02
- 1760753005: [info] Step 1666/1707: training loss=0.00
- 1760753008: [info] Step 1667/1707: training loss=0.00
- 1760753008: [info] Step 1668/1707: training loss=0.00
- 1760753011: [info] Step 1669/1707: training loss=0.00
- 1760753012: [info] Step 1670/1707: training loss=0.61
- 1760753015: [info] Step 1671/1707: training loss=0.00
- 1760753015: [info] Step 1672/1707: training loss=0.00
- 1760753015: [info] Step 1673/1707: training loss=0.00
- 1760753018: [info] Step 1674/1707: training loss=0.00
- 1760753018: [info] Step 1675/1707: training loss=0.00
- 1760753021: [info] Step 1676/1707: training loss=0.05
- 1760753021: [info] Step 1677/1707: training loss=2.36
- 1760753024: [info] Step 1678/1707: training loss=0.00
- 1760753024: [info] Step 1679/1707: training loss=0.00
- 1760753027: [info] Step 1680/1707: training loss=0.00
- 1760753027: [info] Step 1681/1707: training

## Test the Fine-Tuned Model
This is the moment of truth. If the job returned a model ID, I run a quick smoke test to make sure the custom model actually answers. I craft a tiny chat with a system rule that forces emoji-only replies and a blunt user prompt to see how the tone is handled, then call the Chat Completions API with a small token budget and print whatever the model says. If there is no model ID, I explain why and skip the test rather than crashing the notebook. This is not a benchmark or a quality evaluation, just a fast sanity check that the fine-tuning pipeline completed and the model can follow the intended style.

In [8]:
if not fine_tuned_model_id:
    print("Fine-tuned model ID is not available. Cannot run test.")
else:
    test_messages = [
        {
            "role": "system",
            "content": "You're a chatbot that only responds with emojis!",
        },
        {"role": "user", "content": "What the hell is going on?"},
    ]

    print(f"Testing model: {fine_tuned_model_id}")

    completion = client.chat.completions.create(
        model=fine_tuned_model_id,
        messages=test_messages,
        max_tokens=50,
    )

    response_content = completion.choices[0].message.content
    print(f"\nModel Response: {response_content}")

Testing model: ft:gpt-3.5-turbo-0125:personal:emoji-v1:CRqb7Ag7

Model Response: (devil)


## Analysis
I read this output as the model doing exactly what I trained it to do. The assistant targets in my dataset are parenthesized labels rather than Unicode characters, and across 569 examples there are no true emojis in the assistant outputs, with 349 distinct labels that include `(devil)` appearing multiple times. Given that, the reply `(devil)` is a faithful reproduction of the training style and it correctly maps the word hell to a devil sentiment.